# Time Series Forecast Metrics

Forecast evaluation is not one-size-fits-all. A good metric reflects the business cost of errors and
lets you compare models fairly across series and horizons. This notebook focuses on **point-forecast**
metrics and their tradeoffs.

**Metric families**
- Scale-dependent: MAE, RMSE
- Percentage-based: MAPE, sMAPE, WAPE
- Scaled vs naive baseline: MASE

Lower is better for all metrics shown here.


## 1) Build a toy series
We use a synthetic monthly series with trend, seasonality, and noise so the differences between
metrics are easy to see.


In [ ]:
import numpy as np
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
import plotly.io as pio

pio.templates.default = "plotly_white"
rng = np.random.default_rng(42)

def mae(y, yhat):
    y = np.asarray(y)
    yhat = np.asarray(yhat)
    return np.mean(np.abs(y - yhat))

def rmse(y, yhat):
    y = np.asarray(y)
    yhat = np.asarray(yhat)
    return np.sqrt(np.mean((y - yhat) ** 2))

def mape(y, yhat):
    y = np.asarray(y)
    yhat = np.asarray(yhat)
    mask = y != 0
    if not mask.any():
        return np.nan
    return np.mean(np.abs((y[mask] - yhat[mask]) / y[mask])) * 100

def smape(y, yhat):
    y = np.asarray(y)
    yhat = np.asarray(yhat)
    denom = np.abs(y) + np.abs(yhat)
    mask = denom != 0
    if not mask.any():
        return np.nan
    return np.mean(2 * np.abs(y[mask] - yhat[mask]) / denom[mask]) * 100

def wape(y, yhat):
    y = np.asarray(y)
    yhat = np.asarray(yhat)
    denom = np.sum(np.abs(y))
    if denom == 0:
        return np.nan
    return np.sum(np.abs(y - yhat)) / denom * 100

def mase(y, yhat, y_train):
    y = np.asarray(y)
    yhat = np.asarray(yhat)
    y_train = np.asarray(y_train)
    naive_denom = np.mean(np.abs(np.diff(y_train)))
    if naive_denom == 0:
        return np.nan
    return mae(y, yhat) / naive_denom


In [ ]:
n = 120
t = np.arange(n)
trend = 0.2 * t
season = 10 * np.sin(2 * np.pi * t / 12)
noise = rng.normal(0, 3, size=n)
y = 50 + trend + season + noise

index = pd.period_range('2015-01', periods=n, freq='M').to_timestamp()
series = pd.Series(y, index=index, name='y')

train = series.iloc[:-24]
test = series.iloc[-24:]


In [ ]:
fig = go.Figure()
fig.add_trace(go.Scatter(x=train.index, y=train, mode='lines', name='Train'))
fig.add_trace(go.Scatter(x=test.index, y=test, mode='lines', name='Test', line=dict(color='black')))
fig.add_vline(x=test.index[0], line_dash='dash', line_color='gray')
fig.update_layout(title='Synthetic series with train/test split', xaxis_title='Date', yaxis_title='y')
fig

## 2) Baseline forecasts
Simple baselines are strong benchmarks. If your model cannot beat these, it is not useful yet.


In [ ]:
def forecast_naive(train, horizon_index):
    return pd.Series([train.iloc[-1]] * len(horizon_index), index=horizon_index, name='Naive')

def forecast_seasonal_naive(train, horizon_index, season_length=12):
    last_season = train.iloc[-season_length:]
    reps = int(np.ceil(len(horizon_index) / season_length))
    vals = np.tile(last_season.values, reps)[: len(horizon_index)]
    return pd.Series(vals, index=horizon_index, name=f'Seasonal naive (s={season_length})')

def forecast_moving_average(train, horizon_index, window=6):
    mean_val = train.iloc[-window:].mean()
    return pd.Series([mean_val] * len(horizon_index), index=horizon_index, name=f'Moving average (w={window})')

pred_naive = forecast_naive(train, test.index)
pred_seasonal = forecast_seasonal_naive(train, test.index, season_length=12)
pred_ma = forecast_moving_average(train, test.index, window=6)


In [ ]:
fig = go.Figure()
fig.add_trace(go.Scatter(x=train.index, y=train, mode='lines', name='Train', line=dict(color='gray')))
fig.add_trace(go.Scatter(x=test.index, y=test, mode='lines', name='Test', line=dict(color='black')))
fig.add_trace(go.Scatter(x=test.index, y=pred_naive, mode='lines', name=pred_naive.name))
fig.add_trace(go.Scatter(x=test.index, y=pred_seasonal, mode='lines', name=pred_seasonal.name))
fig.add_trace(go.Scatter(x=test.index, y=pred_ma, mode='lines', name=pred_ma.name))
fig.add_vline(x=test.index[0], line_dash='dash', line_color='gray')
fig.update_layout(title='Baseline forecasts on the test horizon', xaxis_title='Date', yaxis_title='y')
fig

## 3) Metric definitions and intuition
Let $y_t$ be the actual value and $\hat{y}_t$ the forecast.

**Scale-dependent**
- MAE: $\text{MAE} = \frac{1}{n} \sum_{t=1}^n |y_t - \hat{y}_t|$
- RMSE: $\text{RMSE} = \sqrt{\frac{1}{n} \sum_{t=1}^n (y_t - \hat{y}_t)^2}$

**Percentage-based**
- MAPE: $\text{MAPE} = \frac{100}{n} \sum_{t=1}^n \left|\frac{y_t - \hat{y}_t}{y_t}\right|$
- sMAPE: $\text{sMAPE} = \frac{100}{n} \sum_{t=1}^n \frac{2|y_t - \hat{y}_t|}{|y_t| + |\hat{y}_t|}$
- WAPE: $\text{WAPE} = 100 \cdot \frac{\sum |y_t - \hat{y}_t|}{\sum |y_t|}$

**Scaled vs naive**
- MASE: $\text{MASE} = \frac{\text{MAE}}{\frac{1}{n-1} \sum_{t=2}^n |y_t - y_{t-1}|}$

**Intuition**
- MAE is the typical absolute miss.
- RMSE punishes large errors (outliers) more than MAE.
- MAPE is easy to explain but breaks when $y_t = 0$.
- sMAPE reduces zero issues and is symmetric.
- WAPE is a stable percentage alternative for many series.
- MASE < 1 means you beat a naive one-step baseline.


In [ ]:
def compute_metrics(y_true, y_pred, y_train):
    return {
        'MAE': mae(y_true, y_pred),
        'RMSE': rmse(y_true, y_pred),
        'MAPE (%)': mape(y_true, y_pred),
        'sMAPE (%)': smape(y_true, y_pred),
        'WAPE (%)': wape(y_true, y_pred),
        'MASE': mase(y_true, y_pred, y_train),
    }

metrics = pd.DataFrame({
    pred_naive.name: compute_metrics(test.values, pred_naive.values, train.values),
    pred_seasonal.name: compute_metrics(test.values, pred_seasonal.values, train.values),
    pred_ma.name: compute_metrics(test.values, pred_ma.values, train.values),
}).T

metrics.round(3)


In [ ]:
scale_dependent = ['MAE', 'RMSE']
scale_free = ['MAPE (%)', 'sMAPE (%)', 'WAPE (%)', 'MASE']

metrics_long = metrics.reset_index().rename(columns={'index': 'model'})

fig1 = px.bar(
    metrics_long.melt(id_vars='model', value_vars=scale_dependent),
    x='model',
    y='value',
    color='variable',
    barmode='group',
    title='Scale-dependent metrics (lower is better)',
)
fig1.update_layout(xaxis_title='Model', yaxis_title='Metric value')
fig1.show()

fig2 = px.bar(
    metrics_long.melt(id_vars='model', value_vars=scale_free),
    x='model',
    y='value',
    color='variable',
    barmode='group',
    title='Scale-free metrics (lower is better)',
)
fig2.update_layout(xaxis_title='Model', yaxis_title='Metric value')
fig2.show()


## 4) Sensitivity checks
Metrics behave differently under scaling and outliers. These small experiments make the tradeoffs
obvious.


In [ ]:
scale = 10
scaled = pd.DataFrame({
    'Original': compute_metrics(test.values, pred_seasonal.values, train.values),
    'Scaled x10': compute_metrics(test.values * scale, pred_seasonal.values * scale, train.values * scale),
}).T

scaled[['MAE', 'RMSE', 'MAPE (%)', 'sMAPE (%)', 'WAPE (%)', 'MASE']].round(3)


In [ ]:
test_outlier = test.copy()
test_outlier.iloc[-1] = test_outlier.iloc[-1] + 30

outlier_effect = pd.DataFrame({
    'Baseline': compute_metrics(test.values, pred_seasonal.values, train.values),
    'With outlier': compute_metrics(test_outlier.values, pred_seasonal.values, train.values),
}).T

outlier_effect[['MAE', 'RMSE', 'MAPE (%)', 'sMAPE (%)', 'WAPE (%)', 'MASE']].round(3)


## 5) Metric selection cheat sheet
- Use **MAE** when errors have roughly linear business cost.
- Use **RMSE** when large misses are disproportionately expensive.
- Use **MASE** to compare across series with different scales.
- Use **sMAPE/WAPE** for percentage-style reporting without MAPE zero problems.
- Avoid **MAPE** if zeros or tiny values are common.

A good practice is to report one scale-dependent metric (MAE or RMSE) plus one scale-free metric
(MASE, sMAPE, or WAPE).


## 6) Extensions and missing pieces
Point metrics are only part of the story. The following topics are natural next notebooks:
- Probabilistic forecasts: pinball loss, CRPS, log score
- Prediction intervals: coverage, Winkler score
- Multi-horizon aggregation and weighting
- Hierarchical and grouped series metrics
- Business-weighted loss functions (cost matrices)
